### Import

In [5]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

Numpy version: 1.26.4
Scikit-learn version: 1.5.1
Pandas version: 1.5.3
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [6]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [7]:
import sys
import os
import pandas as pd

# 현재 작업 디렉토리 경로를 가져와 shared codes 폴더의 위치를 sys.path에 추가합니다.
# sys.path에 추가된 경로에 있는 py 폴더는 임포트할 수 있다.
current_dir = os.getcwd()
shared_codes_dir = os.path.join(current_dir, '../shared codes')
sys.path.append(shared_codes_dir)


# cover_nan 모듈을 임포트
from cover_nan import missing_value_removal_function

# 원본 train 데이터 로드
train = pd.read_csv("../shared codes/data/train.csv")
test = pd.read_csv("../shared codes/data/test.csv")

# missing_value_removal_function 사용
train = missing_value_removal_function(train)
test = missing_value_removal_function(test)

✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!
✅ '대리모 여부' 결측값을 최빈값 (0.0) 으로 대체 완료!
✅ 컬럼 삭제 완료: ['PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '난자 채취 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '난자 혼합 경과일' 결측값을 중앙값 (0.0) 으로 대체 완료!
✅ '배아 이식 경과일' 결측값을 중앙값 (3.0) 으로 대체 완료!


### Data Pre-processing

In [8]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [9]:
# 각 열의 결측값 개수 확인
missing_values_count = X.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)

결측값이 있는 열과 개수:
Series([], dtype: int64)


In [10]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('알 수 없음').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ '알 수 없음' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ '알 수 없음' 값을 포함하는 컬럼들:
['시술 당시 나이', '배란 유도 유형', '난자 기증자 나이', '정자 기증자 나이']


In [11]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('nan').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ 'nan' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ 'nan' 값을 포함하는 컬럼들:
[]


In [12]:
# NaN 값이 존재하는 컬럼 찾기
unknown_columns = X.columns[X.isna().any()].tolist()

unknown_columns

[]

In [13]:
# Categorical(범주형) 칼럼 찾기
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 결과 출력
print("📌 Categorical(범주형) 칼럼 리스트:")
print(categorical_columns)


📌 Categorical(범주형) 칼럼 리스트:
['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이']


In [14]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [15]:
columns_to_drop = [
        "남성 주 불임 원인",
        "남성 부 불임 원인",
        "불임 원인 - 정자 농도",
        "불임 원인 - 정자 면역학적 요인",
        "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태",
        '정자 기증자 나이',
        '배란 유도 유형'
]
X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)  

In [16]:
# 각 열의 결측값 개수 확인
missing_values_count = X_train_encoded.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)


결측값이 있는 열과 개수:
Series([], dtype: int64)


In [17]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 53 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                256351 non-null  float64
 1   시술 시기 코드          256351 non-null  float64
 2   시술 당시 나이          256351 non-null  float64
 3   시술 유형             256351 non-null  float64
 4   특정 시술 유형          256351 non-null  float64
 5   배란 자극 여부          256351 non-null  int64  
 6   단일 배아 이식 여부       256351 non-null  float64
 7   여성 주 불임 원인        256351 non-null  int64  
 8   여성 부 불임 원인        256351 non-null  int64  
 9   부부 주 불임 원인        256351 non-null  int64  
 10  부부 부 불임 원인        256351 non-null  int64  
 11  불명확 불임 원인         256351 non-null  int64  
 12  불임 원인 - 난관 질환     256351 non-null  int64  
 13  불임 원인 - 남성 요인     256351 non-null  int64  
 14  불임 원인 - 배란 장애     256351 non-null  int64  
 15  불임 원인 - 여성 요인     256351 non-null  int64  
 16  불임 원인 - 자궁경부 문제   25

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 데이터 정규화 (X_train_encoded & X_test_encoded)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)  # 동일한 스케일 적용

# DataFrame 변환 (Feature 이름 유지)
feature_names = [f"Feature_{i}" for i in range(X_train_scaled.shape[1])]
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

# 상관 행렬 계산
correlation_matrix_train = X_train_scaled_df.corr()

# 다중 공선성이 높은 칼럼 찾기 (절대 상관 계수가 0.8 이상)
threshold = 0.8
high_corr_features = set()

for i in range(len(feature_names)):
    for j in range(i + 1, len(feature_names)):
        if abs(correlation_matrix_train.iloc[i, j]) > threshold:
            high_corr_features.add(feature_names[j])  # 공선성이 높은 컬럼 추가

# 다중 공선성이 높은 컬럼 제거
X_train_encoded = X_train_scaled_df.drop(columns=high_corr_features, errors='ignore')
X_test_encoded = X_test_scaled_df.drop(columns=high_corr_features, errors='ignore')


In [19]:
X_train_encoded

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_10,...,Feature_39,Feature_42,Feature_43,Feature_44,Feature_46,Feature_48,Feature_49,Feature_50,Feature_51,Feature_52
0,-1.732044,1.518507,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
1,-1.732031,1.020975,2.655919,0.158613,-0.962962,-1.836376,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
2,-1.732017,0.025910,-0.935575,0.158613,0.672585,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.773571
3,-1.732004,-0.471622,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
4,-1.731990,0.025910,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256346,1.731990,1.020975,-0.935575,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,1.142081
256347,1.732004,1.020975,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
256348,1.732017,0.025910,-0.217277,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-0.135020
256349,1.732031,1.518507,0.501022,0.158613,-0.962962,0.544551,-0.543057,-0.178038,-0.112199,-0.094036,...,-0.083423,-0.220261,0.256018,0.344047,-0.430784,-0.098393,-0.0641,0.0,-0.042922,-1.412122


### Train

In [20]:
from imblearn.over_sampling import SMOTE

# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_encoded, y = smote.fit_resample(X_train_encoded, y)

In [21]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

**앙상블 및 Optuna**

In [22]:
import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [23]:
def optimize_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }
    model = XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
    return np.mean(scores)

In [24]:
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0)
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)

In [25]:
def optimize_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)


In [26]:
# Optuna 스터디 실행
print("Optimizing XGBoost...")
xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(optimize_xgboost, n_trials=30)
best_xgb_params = xgb_study.best_params

print("Optimizing LightGBM...")
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(optimize_lightgbm, n_trials=30)
best_lgbm_params = lgbm_study.best_params

print("Optimizing RandomForest...")
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(optimize_random_forest, n_trials=30)
best_rf_params = rf_study.best_params

[I 2025-02-13 21:09:03,652] A new study created in memory with name: no-name-859623c9-fda8-45e9-83fa-2ae6e9a5f554


Optimizing XGBoost...


c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:03] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:06] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:09] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:11] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

Optimizing LightGBM...
[LightGBM] [Info] Number of positive: 152098, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010510 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6234
[LightGBM] [Info] Number of data points in the train set: 304196, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 152099, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6220
[LightGBM] [Info] Number of data points in the train set: 304197, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: p

[I 2025-02-13 21:18:57,304] Trial 0 finished with value: 0.4757748720286905 and parameters: {'n_estimators': 277, 'max_depth': 12, 'learning_rate': 0.16606825698750302, 'num_leaves': 48, 'subsample': 0.8735894117755366}. Best is trial 0 with value: 0.4757748720286905.


...

[I 2025-02-13 21:23:06,115] Trial 28 finished with value: 0.44154570450624525 and parameters: {'n_estimators': 117, 'max_depth': 7, 'learning_rate': 0.18822624499891993, 'num_leaves': 28, 'subsample': 0.7935876518360362}. Best is trial 14 with value: 0.5630975712216542.


[LightGBM] [Info] Number of positive: 152098, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6234
[LightGBM] [Info] Number of data points in the train set: 304196, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 152099, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010188 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6220
[LightGBM] [Info] Number of data points in the train set: 304197, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500002 -> initsco

[I 2025-02-13 21:23:18,040] Trial 29 finished with value: 0.5106007491587918 and parameters: {'n_estimators': 470, 'max_depth': 10, 'learning_rate': 0.23933678835655472, 'num_leaves': 47, 'subsample': 0.884815780224057}. Best is trial 14 with value: 0.5630975712216542.
[I 2025-02-13 21:23:18,041] A new study created in memory with name: no-name-09c744f3-88b9-47e6-a314-265aaffaa5b7


Optimizing RandomForest...


[I 2025-02-13 21:26:50,702] Trial 0 finished with value: 0.4481783513101284 and parameters: {'n_estimators': 199, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.4481783513101284.
[I 2025-02-13 21:29:27,553] Trial 1 finished with value: 0.47355139890319853 and parameters: {'n_estimators': 170, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.47355139890319853.
[I 2025-02-13 21:32:23,245] Trial 2 finished with value: 0.48509346931233166 and parameters: {'n_estimators': 256, 'max_depth': 6, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 2 with value: 0.48509346931233166.
[I 2025-02-13 21:34:09,347] Trial 3 finished with value: 0.48723846543379956 and parameters: {'n_estimators': 156, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 3 with value: 0.48723846543379956.
[I 2025-02-13 21:35:35,645] Trial 4 finished with value: 0.5162595258170976 and parameter

In [27]:
# 최적화된 모델 생성
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

In [28]:
# XGBoost 모델의 파라미터 확인
print("XGBoost Best Parameters:")
print(xgb_model.get_params())

# LightGBM 모델의 파라미터 확인
print("\nLightGBM Best Parameters:")
print(lgbm_model.get_params())

# RandomForest 모델의 파라미터 확인
print("\nRandomForest Best Parameters:")
print(rf_model.get_params())

XGBoost Best Parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.7388933894835854, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'gamma': 0.14580601261269263, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.2955812427886471, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 491, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.6948425192973119, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder'

In [29]:
# Soft Voting 앙상블
ensemble_model = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("lgbm", lgbm_model),
        ("rf", rf_model)
    ],
    voting="soft"
)

In [30]:
# 전체 데이터로 학습
ensemble_model.fit(X_train_encoded, y)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [22:26:12] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 190123, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012660 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5862
[LightGBM] [Info] Number of data points in the train set: 380246, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7388933894835854,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            gamma=0.14580601261269263,
                                            grow_policy=None,
                                            importance_type=None,
                                            intera...
                                            n_estimators=491, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.25217865837911674,
                                             max_depth=5, n_estimators=496,
                                             num_leaves=36, random_state=42,
                                             subsample=0.7711289070138857)),
                             ('rf',
                              RandomForestClassifier(max_depth=3,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     n_estimators=132,
                                                     random_state=42))],
                 voting='soft')

### Predict

In [31]:
pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [32]:
sample_submission = pd.read_csv('../../sample_submission.csv')
sample_submission['probability'] = pred_proba

In [33]:
sample_submission.to_csv('./Result/baseline_submit.csv', index=False)

In [34]:
pred_proba

array([0.11746949, 0.10448746, 0.18770913, ..., 0.32780166, 0.37119434,
       0.11895023])

In [35]:
unique, counts = np.unique(sample_submission['probability'], return_counts=True)
print(unique, counts)

[0.05594023 0.05675022 0.05694426 ... 0.86500426 0.87276575 0.87335867] [1 1 1 ... 1 1 1]
